Lesson 3: Reflection and Blogpost Writing

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
llm_config = {"model": "gpt-3.5-turbo"}

In [1]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''

Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unleashing the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI - a leading platform revolutionizing the way we interact with technology. Offering cutting-edge courses and resources, DeepLearning.AI empowers enthusiasts and professionals to master the intricacies of deep learning. From computer vision to natural language processing, the platform equips learners with the tools to explore endless possibilities in AI. Join the thriving community of innovators, upskill in high-demand fields, and unleash your potential with DeepLearning.AI. Embark on a transformative learning journey today!


Adding reflection: Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI - a leading platform revolutionizing the way we interact with technology. Offering cutting-edge courses and resources, DeepLearning.AI empowers enthusiasts and professionals to master the intricacies of deep learning. From computer vision to natural language processing, the platform equips learners with the tools to explore endless possibilities in AI. Join the thriving community of innovators, upskill in high-demand fields, and unleash your potential with DeepLearning.AI. Embark on a transformative learning journey today!

--------------------------------------------------------------------------------
Critic (to Wr

Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

Orchestrate the nested chats to solve the task

In [12]:
#list of four chats with each reviewer as a recipient 
#later we append this to critic


def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
#Whenever writer sends a message to critic, it triggers nested chats 
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unleashing the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI - a leading platform revolutionizing the way we interact with technology. Offering cutting-edge courses and resources, DeepLearning.AI empowers enthusiasts and professionals to master the intricacies of deep learning. From computer vision to natural language processing, the platform equips learners with the tools to explore endless possibilities in AI. Join the thriving community of innovators, upskill in high-demand fields, and unleash your potential with DeepLearning.AI. Embark on a transformative learning journey today!

--------------------------------------------------------------------------------

************

In [15]:
print(res.summary)

Title: Unleashing the Power of DeepLearning.AI: Exploring Cutting-Edge AI Courses

Delve into the realm of artificial intelligence with DeepLearning.AI, a premier platform offering a myriad of deep learning courses and AI resources. From computer vision to natural language processing, enthusiasts and professionals can master high-demand skills. Join a vibrant community of innovators and unlock the potential of AI. Discover endless possibilities with DeepLearning.AI's transformative learning approach today!

Meta Description: Explore top-notch artificial intelligence courses on DeepLearning.AI and embark on a transformative journey into the world of deep learning. Join a thriving community of innovators and professionals, and unlock the full potential of AI. 

Remember, incorporating keywords like "deep learning" and "artificial intelligence courses" enhances search visibility while adding internal links to related content promotes further exploration. Cheers to expanding your AI horizo